# Benchmark of different methods and languages

Say:

$$\mathrm{sum}(a) = \Sigma_{i=1}^na_i$$

where $n$ is the length of vector $\mathbf{a}$.

In [1]:
n = 10_000_000
a = rand(n)
sum(a)

4.998132558111681e6

In [2]:
@time sum(a)  # usually you should run several times to have a stable time usage.
# That's why it is better to use package BenchmarkTools 

  0.004384 seconds (1 allocation: 16 bytes)


4.998132558111681e6

In [3]:
using BenchmarkTools
@benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.928 ms (0.00% GC)
  median time:      3.069 ms (0.00% GC)
  mean time:        3.054 ms (0.00% GC)
  maximum time:     4.248 ms (0.00% GC)
  --------------
  samples:          1637
  evals/sample:     1

## 1. Julia Built-in

In [4]:
@which sum(a)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:719

In [5]:
# Note there is a link in the result of above cell.
# One can follow the link to learn Julia programming, by watching how others program.
d = Dict()
j_bench = @benchmark sum($a)
d["Julia built-in"] = minimum(j_bench.times)/1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 2.96003

## 2. Julia (hand-written)

In [6]:
function mysum(A)
    s = 0.
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [8]:
j_bench_hand = @benchmark mysum($a)
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6

7.181222

Above, the hand-written function, is about 2x slower than the built-in funciton.

## 3. The C language

In [9]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X){
        double s = 0.;
        for(size_t i=0; i<n; ++i){
            s += X[i];
        }
        return s;
    }
"""
const Clib = tempname()
# compile to a shared library by piping c_code to gcc
open(`gcc -fPIC -O3 -mavx2 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [10]:
c_sum(a) ≈ sum(a) # note here it is a \approx(≈) not =, for round error reason.

true

In [11]:
c_bench = @benchmark c_sum($a)
d["C"] = minimum(c_bench.times) / 1e6

6.925192

## 4. Python's built in sum
The `PyCall` package provides a Julia interface to Python

In [12]:
using PyCall

In [15]:
pysum = pybuiltin("sum")
py_list_bench = @benchmark $pysum($a)
println(pysum(a) ≈ sum(a))
d["Python built-in"] = minimum(py_list_bench.times) / 1e6

true


1523.649625

## 5. Python: `numpy`

In [16]:
numpy_sum = pyimport("numpy")["sum"]
py_numpy_bench = @benchmark $numpy_sum($a)
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6

3.385263

## 6. Python, hand-written

In [17]:
py"""
def py_sum(A):
    s = 0.
    for a in A:
        s += a
    return s
"""
sum_py = py"py_sum"

PyObject <function py_sum at 0x7f58e9aa1af0>

In [18]:
py_hand = @benchmark $sum_py($a)
d["Python hand-written"] = minimum(py_hand.times) / 1e6

1746.412078

## Summary so far

In [21]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, '.'), lpad(round(value; digits=1), 6, '.'))
end

Julia built-in..............3.0
Python numpy................3.4
C...........................6.9
Julia hand-written..........7.2
Python built-in..........1523.6
Python hand-written......1746.4


## Exploiting parallelism with Julia
Why hand written Julia codes are 2x slower?
## 7. Julia (allowing floating point associativity)

In [22]:
function mysum_fast(A)
    s = 0.
    for a in A
        @fastmath s += a
    end
    return s
end

mysum_fast (generic function with 1 method)

In [23]:
j_bench_hand_fast = @benchmark mysum_fast($a)
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6

2.729371

## 8. Distributed Julia (built-in)
or, using more threads.

In [24]:
using Distributed
using DistributedArrays
addprocs(4) # plus the current one, there will have 5 threads available
#@sync @everywhere workers() include("path-to-julia/etc/startup.jl")
@everywhere using DistributedArrays

┌ Info: Precompiling DistributedArrays [aaf54ef3-cdf8-58ed-94cc-d582ad619b94]
└ @ Base loading.jl:1323


In [25]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6

1.487888

## 9. Distributed Julia (hand-written)
you can see more clear how parallel sum works on the same array.

In [26]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

mysum_dist (generic function with 1 method)

In [27]:
j_bench_hand_dist = @benchmark mysum_dist($adist)
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6

1.561696

## Overall Summary
We can't see some 4x faster results is maybe because the CPU is already very fast.  Therefore, the overhead to load the code into CPU played a big part.

In [28]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, '.'), lpad(round(value; digits=1), 6, '.'))
end

Julia 4x built-in...........1.5
Julia 4x hand-written.......1.6
Julia hand-written fast.....2.7
Julia built-in..............3.0
Python numpy................3.4
C...........................6.9
Julia hand-written..........7.2
Python built-in..........1523.6
Python hand-written......1746.4
